---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [9]:
# Use this dictionary to map state names to two letter acronyms
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [10]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. 
    
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/LWNe25tlEemIMA5oe2ze5A
    https://www.coursera.org/learn/python-data-analysis/resources/XlMzq
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/QRjk3bKzEee5Ew6CaivGLg
    https://stackoverflow.com/questions/41457322/pandas-rearranging-a-data-frame/41458629#41458629
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/ZZ8Ri6CGQpafEYughkKWfQ
    '''
    
    
    raw = pd.read_csv('university_towns.txt', sep='\n', header=None)
    raw.rename(columns ={list(raw)[0]: 'data'}, inplace=True)

    ut = pd.DataFrame()

    #print(raw.iloc[:,:])

    #ut['Region'] = raw['data']
    #NOTE: need to use the .split()[0] to access the first element
    raw['RegionName'] = raw['data'].apply(lambda x: x.split(' (')[0].strip() if x.count(' (')>0 else x 
                                          if x.count(',') > 0 else x if x.count(':') > 0 else np.NaN)
    raw['State'] = raw['data'].apply(lambda x: x.split('[edit')[0].strip() 
                                    if x.count('[edit')>0 else np.NaN).fillna(method='ffill')
    raw = raw.dropna().drop('data', axis=1).reindex(columns=['State', 'RegionName']).reset_index(drop=True)


    return raw

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [3]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
    https://codereview.stackexchange.com/questions/213500/finding-recessions-in-us-gdp-data-using-pandas
    '''
    
    raw = pd.read_excel('gdplev.xls', header = [5], parse_cols="E:G").dropna()
    raw = raw[211:] # exclude unneeded data
    raw.index.name = 'Quarter'# name index
    transform = raw # begin data transformation
    transform.rename(columns={'GDP in billions of current dollars': 'GDP in billions', \
                          'GDP in billions of chained 2009 dollars': 'GDP in 09$s'}, inplace = True)
#     transform['qtr_chg'] = transform['GDP in billions of current dollars'] \
#         - transform['GDP in billions of current dollars'].shift(1)
    transform['qtr_chg'] = transform['GDP in 09$s'].pct_change()
    #recession = ('qtr_chg' < 0) & ('qtr_chg'.shift(-1) < 0)
    transform['p_qtr_chg'] = transform['qtr_chg'].shift(1)
   
    list = []
    for index, row in transform.iterrows():
    #print(row['qtr_chg'], row['p_qtr_chg'])
        if row['qtr_chg'] <0 and row['p_qtr_chg'] < 0:
            list.append('yes')
            #print(index)
        else: 
            list.append('no')
    transform['rec'] = list
    trans_filter = transform[transform['rec']=='yes']

    l = trans_filter.index.values.tolist()

    a = transform.index.get_loc(l[0]) - 1# '2009q1'
    b = transform.iloc[a].name


    return b

# o = get_recession_start()

# o



In [4]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    raw = pd.read_excel('gdplev.xls', header = [5], parse_cols="E:G").dropna()
    raw = raw[211:] # exclude unneeded data
    raw.index.name = 'Quarter'# name index
    transform = raw # begin data transformation
    transform.rename(columns={'GDP in billions of current dollars': 'GDP in billions', \
                          'GDP in billions of chained 2009 dollars': 'GDP in 09$s'}, inplace = True)

    transform['qtr_chg'] = transform['GDP in 09$s'].pct_change()
    #recession = ('qtr_chg' < 0) & ('qtr_chg'.shift(-1) < 0)
    transform['p_qtr_chg'] = transform['qtr_chg'].shift(1)
    transform['p_p_qtr_chg'] = transform['p_qtr_chg'].shift(1)

    list = []
    for index, row in transform.iterrows():
    #print(row['qtr_chg'], row['p_qtr_chg'])
        if row['qtr_chg'] >0 and row['p_qtr_chg'] < 0 and row['p_p_qtr_chg'] < 0:
            list.append('yes')
            #print(index)
        else: 
            list.append('no')
    transform['rec_end'] = list


    trans_filter = transform[transform['rec_end']=='yes']

    l = trans_filter.index.values.tolist()
    
    a = transform.index.get_loc(l[0]) + 1# '2009q1'
    b = transform.iloc[a].name
    
    
    
    return b

# z = get_recession_end()
# z


In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    raw = pd.read_excel('gdplev.xls', header = [5], parse_cols="E:G").dropna()
    raw = raw[211:] # exclude unneeded data
    raw.index.name = 'Quarter'# name index
    transform = raw # begin data transformation
    transform.rename(columns={'GDP in billions of current dollars': 'GDP in billions', \
                          'GDP in billions of chained 2009 dollars': 'GDP in 09$s'}, inplace = True)

    transform['qtr_chg'] = transform['GDP in 09$s'].pct_change()
    #recession = ('qtr_chg' < 0) & ('qtr_chg'.shift(-1) < 0)
    transform['p_qtr_chg'] = transform['qtr_chg'].shift(1)


    list = []
    for index, row in transform.iterrows():
    #print(row['qtr_chg'], row['p_qtr_chg'])
        if row['qtr_chg'] <0 and row['p_qtr_chg'] < 0:
            list.append('yes')
            #print(index)
        else: 
            list.append('no')
    transform['rec'] = list
    trans_filter = transform[transform['rec']=='yes']

    a = trans_filter['GDP in 09$s'].idxmin()

    return a

b = get_recession_bottom()
b

TypeError: read_excel() got an unexpected keyword argument 'parse_cols'

In [6]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/nB8Rqba_EeeVOgq1E_eRMA
    
    APPROACH:
    1. discover types of column headings
    2. Convert column headings to dates if not already
    3. combine months into quarters while aggregating data
    4. Convert column names back to date time
    - set index
    
    How to pivot columns to rows?
    
    
    '''
    # read data
    raw = pd.read_csv('City_Zhvi_AllHomes.csv')
    transform = raw
    
    #select the months w/ numbers to aggregate
    transform_dt = transform[transform.columns[6:251]].rename(columns=pd.to_datetime)

    # create the columns as an index
    cols = transform_dt.columns
    sel_cols = cols[(cols >= '2000-01-01')]
    transform_dt = transform_dt[sel_cols].resample('Q',axis=1).mean().rename(
        columns=lambda x: '{:}q{:}'.format(x.year, x.quarter))
    transform_head = transform[transform.columns[0:6]]
    transform_head['StName'] = transform['State'].map(states)
    transform_head.rename(columns={'State':'State_abrv', 'StName':'State'}, inplace=True)
    # for col in transform_dt.columns:
    #     print(type(col)) 
    new_head = transform_head[['State', 'RegionName']]
    #nw_data = pd.concat([new_head, transform_dt])

    new = pd.merge(new_head, transform_dt, how='inner', left_index=True, right_index=True)
    new = new.set_index(['State', 'RegionName'])
    
    return new



In [7]:
# house is quarterly mean value of housing prices in university towns.
house = convert_housing_data_to_quarters()
ut = get_list_of_university_towns()
r_start = get_recession_start()
r_bottom = get_recession_bottom()
r_end = get_recession_end()
ut.info()

<ipython-input-6-ec5a744dc089>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transform_head['StName'] = transform['State'].map(states)
C:\Users\reson\anaconda3\envs\working_env\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


TypeError: read_excel() got an unexpected keyword argument 'parse_cols'

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).
    
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/n6epwLCKEeewNAofllqCYg
    tests
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/Lfa85P5dRmO2vOT-XSZjGA
    
    '''
    from scipy import stats
    house.head()
    a = ut[(ut['State']=='New York') & (ut['RegionName']=='Alfred')]

    data = house[[r_start, r_bottom]]
    data['price_ratio'] = (data['2009q2']/data['2008q3'])-1
    ut_list = ut.to_records(index=False).tolist()

    data_IsUtown = data.loc[ut_list]
    data_NotUtown = data.loc[~data.index.isin(ut_list)]


    #data_IsUtown.info()
    #data_IsUtown['change']
    ttest = stats.ttest_ind(data_IsUtown['price_ratio'], data_NotUtown['price_ratio'], nan_policy='omit')
    Utown_pChg = data_IsUtown['price_ratio'].mean()
    NotUtown_pChg = data_NotUtown['price_ratio'].mean()
    p = ttest[1]

    if p < .01:
        different = True
    else: 
        different = False

    if Utown_pChg >= NotUtown_pChg:
        better = "university town"
    else:
        better = "non-university town"

    r = ()
    r = different, p, better
    r

    return r

# b = run_ttest()
# b

